In [44]:
import pandas as pd
import numpy as np
import nfl_data_py as nfl

In [45]:
df = pd.read_csv('./Data/scrapeddraft0522.csv',index_col=0)

In [46]:
df = df[df.player != '<th aria-label="Player" class="sort_default_asc show_partial_when_sorting left" data-stat="player">Player</th>']

In [47]:
df.player[df.player.str.contains('=')] = df.player[df.player.str.contains('=')].apply(lambda x: str(x).split(">")[1].split("<")[0])

In [48]:
df[['team','round','pick']] = df['draft_info'].apply(lambda x: pd.Series(str(x).split('/')[0:3]))

In [49]:
df['round'] = df['round'].apply(lambda x: pd.Series(str(x).strip()))

In [50]:
df = df[df['round'] != '']

In [51]:
df['round'] = df['round'].apply(lambda x: pd.Series(str(x)[0]))

In [52]:
df['round'][df['round']=='n'] = 8

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\1221531417.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['round'][df['round']=='n'] = 8


In [53]:
df['pick'] = df['pick'].apply(lambda x: pd.Series(str(x).strip().split(' ')[0]))

In [54]:
df['pick'] = df['pick'].apply(lambda x: pd.Series(str(x)[0:-2]))

In [55]:
df['pick'][df['pick'] == ''] = np.nan

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\1104288695.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pick'][df['pick'] == ''] = np.nan


In [56]:
df[['feet','inches']] = df['height'].apply(lambda x: pd.Series(str(x).split('-')))

In [57]:
df['feet'] = pd.to_numeric(df.feet,errors='coerce')
df['inches'] = pd.to_numeric(df.inches,errors='coerce')
df['height'] = (df.feet * 12) + df.inches

In [58]:
df.drop(['inches','feet'],axis=1,inplace=True)

In [59]:
df.drop(['link','draft_info'],axis = 1, inplace=True)

In [60]:
# Call NFL Players Python pacakge
player = nfl.import_players()

# Find all players who were drafted in 2005 or later
draft_players = player.loc[(player.entry_year > 2004) & (player.draft_number >= 1)].sort_values(by=['entry_year','draft_number'])

# Select relevant columns of draft players
draft_player = draft_players[['display_name','position_group','entry_year','college_conference','draft_number']]

In [61]:
cat_cols = ['player','pos','school_name','team']

In [62]:
num_cols = [col for col in df if col not in cat_cols]

In [63]:
df[num_cols] = df[num_cols].apply(pd.to_numeric,errors='coerce', axis=1)

In [64]:
drafted = df[df['round'] != 8]

In [65]:
draft = df.merge(draft_player,how='left',left_on=['year','pick'],right_on=['entry_year','draft_number'])

In [66]:
age = pd.read_excel('./Data/Age.xlsx')

In [67]:
age = age[['Year','Rnd','Pick','Age','Player']]

In [68]:
age[['Year','Rnd','Pick','Age']] = age[['Year','Rnd','Pick','Age']].apply(pd.to_numeric,errors='coerce',axis=1)

In [69]:
drafts = draft.merge(age,how='left',left_on=['year','pick','player'],right_on=['Year','Pick','Player'])

In [70]:
drafts.drop(['display_name','position_group','entry_year','draft_number','Year','Rnd','Pick','Player'],axis=1,inplace=True)

In [71]:
drafts.pos[(drafts.pos == 'OLB')|(drafts.pos == 'ILB')] = 'LB'
drafts.pos[(drafts.pos == 'OT')|(drafts.pos == 'C')|(drafts.pos == 'OG')] = 'OL'

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\1009202155.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.pos[(drafts.pos == 'OLB')|(drafts.pos == 'ILB')] = 'LB'
C:\Users\capta\AppData\Local\Temp\ipykernel_21532\1009202155.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.pos[(drafts.pos == 'OT')|(drafts.pos == 'C')|(drafts.pos == 'OG')] = 'OL'


In [72]:
drafts['Round'] = drafts['round']

In [73]:
drafts.drop(['round'],axis=1,inplace=True)

In [74]:
drafts.college_conference[(drafts.college_conference == 'NO COLLEGE') & (drafts.school_name == 'Ohio State')] = 'Big Ten Conference'

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\1151124999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.college_conference[(drafts.college_conference == 'NO COLLEGE') & (drafts.school_name == 'Ohio State')] = 'Big Ten Conference'


In [75]:
drafts.college_conference[(drafts.college_conference == 'NO COLLEGE')] = 'Atlantic Coast Conference'

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\3583540359.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.college_conference[(drafts.college_conference == 'NO COLLEGE')] = 'Atlantic Coast Conference'


In [76]:
drafts.college_conference[(drafts.college_conference == 'NO FOOTBALL')] = 'Conference USA'

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\2083932286.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.college_conference[(drafts.college_conference == 'NO FOOTBALL')] = 'Conference USA'


In [77]:
drafts.college_conference[(drafts.college_conference == 'Pacific Ten Conference')] = 'Pacific Twelve Conference'

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\3775298207.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.college_conference[(drafts.college_conference == 'Pacific Ten Conference')] = 'Pacific Twelve Conference'


In [78]:
drafts['speed'] = (drafts.weight * 200) / (drafts.forty_yd ** 4)
drafts['shuttle_agility'] = (drafts.weight * 200) / (drafts.shuttle ** 4)
drafts['cone_agility'] = (drafts.weight * 200) / (drafts.cone ** 3)

In [79]:
drafts.Round[drafts.Round <= 3] = 2

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\874012890.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.Round[drafts.Round <= 3] = 2


In [80]:
drafts.Round[drafts.Round == 8] = 0

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\2272459413.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.Round[drafts.Round == 8] = 0


In [81]:
drafts.Round[drafts.Round >= 4] = 1

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\254994376.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.Round[drafts.Round >= 4] = 1


In [85]:
drafts

,player,year,pos,school_name,height,weight,forty_yd,vertical,bench_reps,broad_jump,...,fumbles_forced_season,fumbles_forced_career,team,pick,college_conference,Age,Round,speed,shuttle_agility,cone_agility
0,Hamza Abdullah,2005.0,S,Washington State,74.0,214.0,NaN,39.0,18.0,114.0,...,NaN,NaN,Tampa Bay Buccaneers,231.0,NaN,22.0,1.0,NaN,NaN,128.045806
1,Anthony Alabi,2005.0,OL,TCU,77.0,310.0,5.12,32.5,18.0,105.0,...,NaN,NaN,Miami Dolphins,162.0,NaN,24.0,1.0,90.221874,138.471489,140.128598
2,Lorenzo Alexander,2005.0,DT,California,74.0,289.0,5.12,29.5,28.0,106.0,...,NaN,NaN,nan,NaN,NaN,NaN,0.0,84.110070,134.859719,135.376384
3,Jared Allen,2005.0,QB,Florida Atlantic,74.0,218.0,4.93,27.5,NaN,106.0,...,NaN,NaN,nan,NaN,NaN,NaN,0.0,73.807214,117.389272,112.539221
4,Derek Anderson,2005.0,QB,Oregon State,78.0,242.0,5.03,29.0,NaN,103.0,...,NaN,NaN,Baltimore Ravens,213.0,Pacific Twelve Conference,22.0,1.0,75.608987,106.237406,107.265304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6062,Devonte Wyatt,2022.0,DT,Georgia,75.0,304.0,4.77,29.0,NaN,111.0,...,2.0,2.0,Green Bay Packers,28.0,Southeastern Conference,24.0,2.0,117.443791,NaN,NaN
6063,Jalen Wydermyer,2022.0,TE,Texas A&M,76.0,255.0,NaN,NaN,NaN,NaN,...,NaN,NaN,nan,NaN,NaN,NaN,0.0,NaN,NaN,NaN
6064,Cade York,2022.0,K,LSU,73.0,206.0,NaN,NaN,12.0,NaN,...,NaN,NaN,Cleveland Browns,124.0,Southeastern Conference,21.0,1.0,NaN,NaN,NaN
6065,Nick Zakelj,2022.0,OL,Fordham,78.0,316.0,5.13,28.5,27.0,110.0,...,NaN,NaN,San Francisco 49ers,187.0,Patriot League,23.0,1.0,91.253098,128.420232,135.772549


In [82]:
drafts.to_csv('./Data/draft0522.csv')

In [83]:
drafts.to_parquet('./Data/draft0522.parquet')

In [84]:
#drafts.school_name[(drafts.college_conference.isna())].value_counts()